In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## all necessary modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# import breast-cancer data
df=pd.read_csv("/kaggle/input/breast-cancer-wisconsin-data/data.csv",header=0)


In [ ]:
# Display the top 5 records of the data
df.head()

In [ ]:
#  Each features names and its datatype
df.dtypes

In [ ]:
# Finding null values in a each column
df.isnull().sum()

In [ ]:
# removing unnamed:32 feature containing all null values
df.drop(["Unnamed: 32"],axis=1,inplace=True)

In [ ]:
# In order to view some basic statistics of dataframe
df.iloc[:,1:].describe()

In [ ]:
mean_features=['radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean']
se_features=['radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se']
worst_features=['radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst']

In [ ]:
plt.figure(figsize=(10,6))
df["diagnosis"].value_counts().plot(kind="bar")
plt.ylabel("count")

**Here B means Beningn(non cancerous) and M means Malignant(cancerous) **

In [ ]:
# Displaying each features distribution
for feature in mean_features:
    plt.figure(figsize=(10,6))
    sns.distplot(df[feature])
    plt.title(feature+" skew "+str(df[feature].skew()))
    plt.show()

**All features in mean_features category are lightly positive skewed**

In [ ]:
for feature in se_features:
    plt.figure(figsize=(10,6))
    sns.distplot(df[feature])
    plt.title(feature+" skew "+str(df[feature].skew()))
    plt.show()

**All features in se_features category are highly positive skewed then mean_features**

In [ ]:
for feature in worst_features:
    plt.figure(figsize=(10,6))
    sns.distplot(df[feature])
    plt.title(feature+" skew "+str(df[feature].skew()))
    plt.show()

In [ ]:
# target feature diagnosis vs mean_features
for item in mean_features:
    plt.figure(figsize=(10,6))
    sns.boxplot(x="diagnosis",y=item,data=df)
    plt.title(item)
    plt.show()

**From above box plot we can say that malignant has high median value compared to benign** 

In [ ]:
# target feature diagnosis vs se_features
for item in se_features:
    plt.figure(figsize=(10,6))
    sns.boxplot(x="diagnosis",y=item,data=df)
    plt.title(item)
    plt.show()

**From above box plot we can say that malignant has high median value compared to benign for most of the features**

In [ ]:
for item in worst_features:
    plt.figure(figsize=(10,6))
    sns.boxplot(x="diagnosis",y=item,data=df)
    plt.title(item)
    plt.show()

In [ ]:
# convert target feature diagnosis to banary
le=LabelEncoder()
df["diagnosis"]=le.fit_transform(df["diagnosis"])

In [ ]:
#correlation between diagnosis feature and mean_features
plt.figure(figsize=(20,10))
sns.heatmap(df[["diagnosis"]+mean_features].corr(),annot=True)

**The feature which are correlated with target feature(diagnosis) are radius_mean,texture_mean,perimeter_mean,area_mean,"smoothness_se","compactness_se","symmetry_se",'concavity_mean','concave points_mean'but (radius_mean,texture_mean,perimeter_mean,area_mean) are highly correlated to eachother to consider any one feature i.e.,radius_mean and ("compactness_se",'concavity_mean','concave points_mean) consider one feature "compactness_se"

In [ ]:
final_mean_features=["radius_mean","texture_mean","smoothness_mean","compactness_mean","symmetry_mean"]

In [ ]:
#correlation between diagnosis feature and se_features
plt.figure(figsize=(20,10))
sns.heatmap(df[["diagnosis"]+mean_features].corr(),annot=True)

In [ ]:
final_se_features=["radius_se","texture_se","smoothness_se","compactness_se","symmetry_se"]

In [ ]:
#correlation between diagnosis feature and worst_features
plt.figure(figsize=(20,10))
sns.heatmap(df[["diagnosis"]+mean_features].corr(),annot=True)

In [ ]:
final_worst_features=["radius_worst","texture_worst","smoothness_worst","compactness_worst","symmetry_worst"]

In [ ]:
# scale the final features by using minmaxscaler
mms=MinMaxScaler()
df[final_mean_features]=mms.fit_transform(df[final_mean_features])
df[final_se_features]=mms.fit_transform(df[final_se_features])
df[final_worst_features]=mms.fit_transform(df[final_worst_features])

In [ ]:
x=df[final_mean_features+final_se_features+final_worst_features]
y=df["diagnosis"]

In [ ]:
# splitting the data into train and test data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
def model(x_train,y_train):
    ## LogisticRegression
    lr=LogisticRegression()
    lr.fit(x_train,y_train)
    y_plr=lr.predict(x_test)
    ## DecisionTreeClassifier
    dtc=DecisionTreeClassifier(criterion="entropy")
    dtc.fit(x_train,y_train)
    y_dtc=dtc.predict(x_test)
    ## RandomForestClassifier
    rfc=RandomForestClassifier(n_estimators=40,criterion="entropy")
    rfc.fit(x_train,y_train)
    y_rfc=rfc.predict(x_test)
    ## Supprt vector machines
    svm=SVC(kernel="rbf")
    svm.fit(x_train,y_train)
    y_svm=svm.predict(x_test)
    ## Accuracy
    print("the logistic accuracy is",accuracy_score(y_test,y_plr))
    print("the Decision accuracy is",accuracy_score(y_test,y_dtc))
    print("the Random accuracy is",accuracy_score(y_test,y_rfc))
    print("the svm accuracy is",accuracy_score(y_test,y_svm))
    print("the logistic classification_report is")
    print(classification_report(y_test,y_plr))
    print("the Decision classification_report is")
    print(classification_report(y_test,y_dtc))
    print("the Random classification_report is")
    print(classification_report(y_test,y_rfc))
    print("the svm classification_report is")
    print(classification_report(y_test,y_svm))

In [ ]:
model(x_train,y_train)